### Import required libraries

In [1]:
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential as Seq, Linear, ReLU

from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, Normalizer

In [2]:
!pip install torch_geometric
tv = torch.__version__
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{tv}.html

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cpu.html


In [3]:
from torch_geometric.data import Data
from torch_geometric.nn import DynamicEdgeConv, MessagePassing, knn_graph

### Selecting Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Load traning and test data and Concatinate them

In [35]:
df_train = pd.read_csv('trainingData_mod.csv')

In [36]:
missing = 0.0
if missing > 0:
    df_train = df_train.sample(frac=1-missing, random_state=42)

In [37]:
df_val = pd.read_csv('validationData_mod.csv')

train_numb = len(df_train)
test_numb = len(df_val)

df = pd.concat([df_train, df_val])
df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,DRSS520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,0,0,0,0,0,0,0,0,0,0,...,0.0,-7541.264300,4.864921e+06,2,1,106,2,2,23,1371713733
1,0,0,0,0,0,0,0,0,0,0,...,0.0,-7536.621200,4.864934e+06,2,1,106,2,2,23,1371713691
2,0,0,0,0,0,0,0,7,0,0,...,0.0,-7519.152400,4.864950e+06,2,1,103,2,2,23,1371714095
3,0,0,0,0,0,0,0,0,0,0,...,0.0,-7524.570400,4.864934e+06,2,1,102,2,2,23,1371713807
4,0,0,0,0,0,0,0,0,0,0,...,0.0,-7632.143600,4.864982e+06,0,0,122,2,11,13,1369909710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,...,0.0,-7317.344231,4.864796e+06,3,2,0,0,0,13,1381156711
1107,0,0,0,0,0,0,0,0,0,0,...,0.0,-7313.731120,4.864792e+06,3,2,0,0,0,13,1381156730
1108,0,0,0,0,0,0,0,0,0,0,...,0.0,-7637.535798,4.864903e+06,0,0,0,0,0,13,1381247781
1109,0,0,0,0,0,0,0,0,0,0,...,0.0,-7636.654005,4.864905e+06,0,0,0,0,0,13,1381247807


### Create Classes for classification

In [38]:
df['CLASS'] = df['BUILDINGID'].astype(str) + df['FLOOR'].astype(str)
df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,CLASS
0,0,0,0,0,0,0,0,0,0,0,...,-7541.264300,4.864921e+06,2,1,106,2,2,23,1371713733,12
1,0,0,0,0,0,0,0,0,0,0,...,-7536.621200,4.864934e+06,2,1,106,2,2,23,1371713691,12
2,0,0,0,0,0,0,0,7,0,0,...,-7519.152400,4.864950e+06,2,1,103,2,2,23,1371714095,12
3,0,0,0,0,0,0,0,0,0,0,...,-7524.570400,4.864934e+06,2,1,102,2,2,23,1371713807,12
4,0,0,0,0,0,0,0,0,0,0,...,-7632.143600,4.864982e+06,0,0,122,2,11,13,1369909710,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,...,-7317.344231,4.864796e+06,3,2,0,0,0,13,1381156711,23
1107,0,0,0,0,0,0,0,0,0,0,...,-7313.731120,4.864792e+06,3,2,0,0,0,13,1381156730,23
1108,0,0,0,0,0,0,0,0,0,0,...,-7637.535798,4.864903e+06,0,0,0,0,0,13,1381247781,00
1109,0,0,0,0,0,0,0,0,0,0,...,-7636.654005,4.864905e+06,0,0,0,0,0,13,1381247807,00


In [39]:
no_out = len(df['CLASS'].unique())

In [40]:
df_y = df[["CLASS"]]
df_y

,CLASS
0,12
1,12
2,12
3,12
4,00
...,...
1106,23
1107,23
1108,00
1109,00


In [41]:
enc = OrdinalEncoder(dtype=np.int32)
y = enc.fit_transform(df_y['CLASS'].values.reshape(-1,1))
y = y.reshape(-1,)

In [42]:
y.max()

12

In [43]:
y

array([6, 6, 6, ..., 0, 0, 0])

### Seperate Input and Output

In [45]:
df_x = df.iloc[: , :1040]

df_x

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,DRSS511,DRSS512,DRSS513,DRSS514,DRSS515,DRSS516,DRSS517,DRSS518,DRSS519,DRSS520
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,7,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1107,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1108,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1109,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# Split data into train and test sets
X_train = df_x[:len(df_train)]
X_test = df_x[len(df_train):]
y_train = y[:len(df_train)]
y_test = y[len(df_train):]

### Scale the input features

In [47]:
norm_scl = Normalizer()

norm_scl.fit(X_train)

Normalizer()

In [48]:
df_x = norm_scl.transform(df_x)

In [49]:
X_train = norm_scl.transform(X_train)
X_test = norm_scl.transform(X_test)

In [50]:
print("Normalized X_train:")
print(X_train[0])

Normalized X_train:
[0. 0. 0. ... 0. 0. 0.]


In [51]:
# Define the node features
x = torch.tensor(df_x, dtype=torch.float).float()

# Define the node labels
y = torch.tensor(y, dtype=torch.float).long().reshape(-1)

# Create the Data object
data = Data(x=x, y=y)
data = data.to(device)

In [52]:
# train_mask = np.array([random.random() < 0.8 for _ in data.x])
train_mask = torch.BoolTensor([True]*(train_numb) + [False]*test_numb).view(-1)
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [53]:
test_mask = np.invert(train_mask)
test_mask

tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.uint8)

In [54]:
data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

C:\Users\rohil\AppData\Local\Temp\ipykernel_17188\950961007.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
C:\Users\rohil\AppData\Local\Temp\ipykernel_17188\950961007.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.test_mask = torch.tensor(test_mask, dtype=torch.bool)


In [55]:
# number of nodes
print("Number of nodes: ", data.num_nodes)

Number of nodes:  21048


In [56]:
# sample nodes from the graph
print("Shape of sample nodes: ", data.x.shape)

Shape of sample nodes:  torch.Size([21048, 1040])


In [57]:
# check training nodes
print("# of nodes to train on: ", data.train_mask.sum().item())

# check test nodes
print("# of nodes to test on: ", data.test_mask.sum().item())

# of nodes to train on:  19937
# of nodes to test on:  1111


In [58]:
print("X shape: ", data.x.shape)
# print("Edge shape: ", data.edge_index.shape)
print("Y shape: ", data.y.shape)

X shape:  torch.Size([21048, 1040])
Y shape:  torch.Size([21048])


In [59]:
class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='sum')
        self.mlp = Seq(Linear(2 * in_channels, out_channels), ReLU(), Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [60]:
class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=5):
        super().__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

### Define Model

In [61]:
class DEC(torch.nn.Module):
    def __init__(self, k1, k2):
        super().__init__()
        self.conv1 = DynamicEdgeConv(data.num_features, 256, k=k1)
        self.conv2 = DynamicEdgeConv(256, 50, k=k2)
        self.fc1 = nn.Linear(50, no_out)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x = data.x
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

In [62]:
# useful function for computing accuracy
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

In [63]:
data.y

tensor([6, 6, 6,  ..., 0, 0, 0])

### Train the model

In [2]:
def train_fun(k1, k2, lr):
    model = DEC(k1, k2).to(device)
    print("model uploaded")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    loss_function = nn.CrossEntropyLoss()
    
    losses = []
    accuracies = []
    test_accu = []

    # epoch = 0
    for epoch in range(10):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        correct = compute_accuracy(out.argmax(dim=1)[data.train_mask], data.y[data.train_mask])
        acc = int(correct) / int(data.train_mask.sum())
        losses.append(loss.item())
        accuracies.append(acc)


        loss.backward()
        optimizer.step()

        # epoch+=1

        if (epoch) % 1 == 0:
            model.eval()
            pred = model(data).argmax(dim=1)
            correct = compute_accuracy(pred[data.test_mask], data.y[data.test_mask])
            acc_test = int(correct) / int(data.test_mask.sum())
            test_accu.append(acc_test)
            print(f'Epoch: {epoch}, \tAccuracy: {acc}, \tTest accuracy: {acc_test}')
          
    res = f"best performance on train: {max(accuracies)}, test: {max(test_accu)}"  
    print(res)
    plt.plot(accuracies)
    plt.plot(test_accu)
    plt.title(res)
    plt.legend(['Train acc.', 'Test acc.'])
    plt.savefig('with DRSS.png')
    plt.show()

In [3]:
train_fun(10, 6, 0.009)

NameError: name 'DEC' is not defined

### Grid Search with varying k1 and k2

In [ ]:
for k1 in [30, 20, 15, 10, 5, 2]:
    for k2 in [15, 10, 6, 3, 1]:
        if k1 > k2:
            print("Doing for: ", k1, k2)
            train_fun(k1, k2, 0.009)

Doing for:  30 15
